<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/ver049/Team01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.9/83.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.2/234.2 kB 10.1 MB/s eta 0:00:00


In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/tutorial/teams.html


In [3]:
import asyncio

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Create an OpenAI model client.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-2024-08-06",
    # api_key="sk-...", # Optional if you have an OPENAI_API_KEY env variable set.
)

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Respond with 'APPROVE' to when your feedbacks are addressed.",
)

# Define a termination condition that stops the task if the critic approves.
text_termination = TextMentionTermination("APPROVE")

# Create a team with the primary and critic agents.
team = RoundRobinGroupChat([primary_agent, critic_agent], termination_condition=text_termination)


In [4]:
# Use `asyncio.run(...)` when running in a script.
result = await team.run(task="Write a short poem about the fall season.")
print(result)


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=142), metadata={}, content="Leaves of amber, gold, and rust  \nGently flutter, earthward thrust.  \nCrisp air whispers through the trees,  \nA symphony of autumn's breeze.  \n\nPumpkins glow in orange delight,  \nHarvest moon ignites the night.  \nSweaters wrap in cozy embrace,  \nAs scarves and hats find their place.  \n\nAcorns drop with silent grace,  \nA squirrel's treasure, nature's pace.  \nBonfires crackle, stories told,  \nAs daylight dwindles, soft and cold.  \n\nThe world adorned in mellow hues,  \nA tapestry of fall's debut.  \nIn every leaf, a farewell song,  \nAs autumn's magic sweeps along.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=187, completion_tokens=232), metadata={}, conten

In [5]:
# When running inside a script, use a async main function and call it from `asyncio.run(...)`.
await team.reset()  # Reset the team for a new task.
async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
    if isinstance(message, TaskResult):
        print("Stop Reason:", message.stop_reason)
    else:
        print(message)


source='user' models_usage=None metadata={} content='Write a short poem about the fall season.' type='TextMessage'
source='primary' models_usage=RequestUsage(prompt_tokens=28, completion_tokens=104) metadata={} content="In autumn's gentle, golden glow,  \nThe whispers of the breezes flow,  \nLeaves alight in fiery dance,  \nA tapestry of earth's romance.  \n\nThe air turns crisp, a cool embrace,  \nAs pumpkins don their orange grace.  \nFields of amber, skies of slate,  \nThe harvest yields, the year awaits.  \n\nWhispers of the coming rest,  \nNature dons her scarlet vest.  \nIn the quiet of the fall,  \nThe world prepares for winter's call.  " type='TextMessage'
source='critic' models_usage=RequestUsage(prompt_tokens=149, completion_tokens=245) metadata={} content='Your poem beautifully captures the essence of fall with its vivid imagery and gentle rhythm. Here are a few suggestions to enhance it further:\n\n1. **Imagery Consistency**: While you\'ve used excellent imagery, try to mai

In [6]:
await team.reset()  # Reset the team for a new task.
await Console(team.run_stream(task="Write a short poem about the fall season."))  # Stream the messages to the console.


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Leaves of amber gently fall,  
Whispering winds begin their call.  
Golden hues paint the ground,  
In autumn's embrace, life is found.  

Crisp air dances with morning light,  
Harvest moon gleams in the night.  
Nature's quilt of red and gold,  
Tales of warmth in chill retold.  

Pumpkin scents and fires aglow,  
Scarves and sweaters steal the show.  
In fall's gentle, fading grace,  
We find reflection, time and space.  
---------- critic ----------
Your poem beautifully captures the essence of the fall season with vivid imagery and a gentle rhythm. The descriptions of leaves, colors, and scents evoke a warm, nostalgic feeling that resonates with the reader. Here are a few suggestions to enhance it:

1. **Rhythm and Flow:** Consider maintaining a more consistent rhythm or syllable count in each line to create a smoother flow. For example, matching the syllable count in corresponding l

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=108), metadata={}, content="Leaves of amber gently fall,  \nWhispering winds begin their call.  \nGolden hues paint the ground,  \nIn autumn's embrace, life is found.  \n\nCrisp air dances with morning light,  \nHarvest moon gleams in the night.  \nNature's quilt of red and gold,  \nTales of warmth in chill retold.  \n\nPumpkin scents and fires aglow,  \nScarves and sweaters steal the show.  \nIn fall's gentle, fading grace,  \nWe find reflection, time and space.  ", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=153, completion_tokens=215), metadata={}, content='Your poem beautifully captures the essence of the fall season with vivid imagery and a gentle rhythm. The descriptions of leaves, colors, and

In [7]:
await team.reset()  # Reset the team for the next run.


In [8]:
# Create a new team with an external termination condition.
external_termination = ExternalTermination()
team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=external_termination | text_termination,  # Use the bitwise OR operator to combine conditions.
)

# Run the team in a background task.
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

# Wait for some time.
await asyncio.sleep(0.1)

# Stop the team.
external_termination.set()

# Wait for the team to finish.
await run


---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Golden leaves dance upon the breeze,  
Whispering secrets of the trees.  
Crisp air kisses cheeks aglow,  
As autumn's magic starts to show.  

Pumpkins line the winding lane,  
Harvest bounty, nature's gain.  
Days grow shorter, nights embrace,  
A time for warmth, a slower pace.  

Scarlet sunsets, skies aflame,  
Echo the beauty of this name.  
Fall, a season rich and deep,  
In its quiet arms, we gently sleep.  


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=28, completion_tokens=107), metadata={}, content="Golden leaves dance upon the breeze,  \nWhispering secrets of the trees.  \nCrisp air kisses cheeks aglow,  \nAs autumn's magic starts to show.  \n\nPumpkins line the winding lane,  \nHarvest bounty, nature's gain.  \nDays grow shorter, nights embrace,  \nA time for warmth, a slower pace.  \n\nScarlet sunsets, skies aflame,  \nEcho the beauty of this name.  \nFall, a season rich and deep,  \nIn its quiet arms, we gently sleep.  ", type='TextMessage')], stop_reason='External termination requested')

In [ ]:
await Console(team.run_stream())  # Resume the team to continue the last task.


In [10]:
# The new task is to translate the same poem to Chinese Tang-style poetry.
await Console(team.run_stream(task="한글로 써주세요"))


---------- user ----------
한글로 써주세요
---------- primary ----------
물감을 머금은 나뭇잎이 춤추고,  
바람 속 나무들의 비밀을 속삭여요.  
서늘한 바람이 뺨을 스치면,  
가을의 마법이 천천히 드러나죠.  

줄줄이 늘어선 호박들은 황금빛을 띄고,  
풍성한 수확 이야기를 풀어내요.  
낮은 짧아지고 밤은 길어지며,  
따뜻함을 부르는 느긋한 시간이 흐르죠.  

짙어지는 저녁 속에 나무 연기가 퍼지고,  
하늘은 석양 속에 타오르죠.  
가을은 황금색 옷을 두르고,  
평온 속에 우리를 안아주네요.  

이 시가 가을의 정취를 잘 담아내길 바랍니다. 추가로 원하는 수정 사항이 있으면 알려주세요!
---------- critic ----------
이 한글 번역은 원래 시의 감성을 잘 전달하고 있습니다. 가을의 분위기를 아름답게 그려내면서 리듬과 운율도 자연스럽게 이어지는 점이 매우 좋습니다. 추가로 고려할 수 있는 사항이 있다면, 한국의 가을과 관련된 전통적인 이미지나 감성을 조금 더 반영해보는 것도 흥미로울 수 있을 것 같습니다. 하지만 현재로서는 이미 충분히 감명 깊고, 가을의 정취를 잘 담아내고 있는 훌륭한 시입니다.

훌륭한 번역에 감사합니다. 추가 의견이 필요하다면 언제든지 알려주세요!

APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='한글로 써주세요', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=639, completion_tokens=208), metadata={}, content='물감을 머금은 나뭇잎이 춤추고,  \n바람 속 나무들의 비밀을 속삭여요.  \n서늘한 바람이 뺨을 스치면,  \n가을의 마법이 천천히 드러나죠.  \n\n줄줄이 늘어선 호박들은 황금빛을 띄고,  \n풍성한 수확 이야기를 풀어내요.  \n낮은 짧아지고 밤은 길어지며,  \n따뜻함을 부르는 느긋한 시간이 흐르죠.  \n\n짙어지는 저녁 속에 나무 연기가 퍼지고,  \n하늘은 석양 속에 타오르죠.  \n가을은 황금색 옷을 두르고,  \n평온 속에 우리를 안아주네요.  \n\n이 시가 가을의 정취를 잘 담아내길 바랍니다. 추가로 원하는 수정 사항이 있으면 알려주세요!', type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=864, completion_tokens=149), metadata={}, content='이 한글 번역은 원래 시의 감성을 잘 전달하고 있습니다. 가을의 분위기를 아름답게 그려내면서 리듬과 운율도 자연스럽게 이어지는 점이 매우 좋습니다. 추가로 고려할 수 있는 사항이 있다면, 한국의 가을과 관련된 전통적인 이미지나 감성을 조금 더 반영해보는 것도 흥미로울 수 있을 것 같습니다. 하지만 현재로서는 이미 충분히 감명 깊고, 가을의 정취를 잘 담아내고 있는 훌륭한 시입니다.\n\n훌륭한 번역에 감사합니다. 추가 의견이 필요하다면 언제든지 알려주세요!\n\nAPPROVE', type='TextMessage')], stop_